In [1]:
#Importamos la librería
from bs4 import BeautifulSoup
import numpy as np
import requests
import json
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.chrome.options import Options
import tqdm
import time


In [2]:
# Definimos el url de la pagina a consultar
url = "https://www.makro.pe/tiendas"
# Obtenemos el html de la página que queremos consultar
response =  requests.get(url)
# Verificamos si la consulta es correcta, status_code= 200
contenido_pagina = ""
if response.status_code == 200:
    contenido_pagina = response.content
print(contenido_pagina)


b'<!DOCTYPE html><html lang="es"><head><meta charSet="utf-8"/><script id="Cookiebot" src="https://consent.cookiebot.com/uc.js" data-cbid="ab8246b2-4fac-4eaf-8851-354b2f4ba76c" data-blockingmode="auto" type="text/javascript" defer=""></script><title>Tiendas | Makro - Tu mejor aliado</title><meta name="description" content="Tiendas makro"/><meta property="og:type" content="website"/><meta property="og:title" content="Tiendas | Makro - Tu mejor aliado"/><meta property="og:description" content="Tiendas makro"/><meta property="og:site_name" content="Makro"/><meta property="og:url" content="https://www.makro.pe/"/><meta property="og:image" content="https://www.makro.pe/opengraph/default-image.jpeg"/><meta property="og:image:width" content="800"/><meta property="og:image:height" content="600"/><meta property="og:image:alt" content="Makro"/><meta name="twitter:card" content="summary_large_image"/><meta name="twitter:site" content="Makro"/><meta name="twitter:creator" content="Makro"/><meta nam

In [3]:
# Parseamos la estructura con bs4
soup = BeautifulSoup(contenido_pagina, 'html.parser')
soup

<!DOCTYPE html>
<html lang="es"><head><meta charset="utf-8"/><script data-blockingmode="auto" data-cbid="ab8246b2-4fac-4eaf-8851-354b2f4ba76c" defer="" id="Cookiebot" src="https://consent.cookiebot.com/uc.js" type="text/javascript"></script><title>Tiendas | Makro - Tu mejor aliado</title><meta content="Tiendas makro" name="description"/><meta content="website" property="og:type"/><meta content="Tiendas | Makro - Tu mejor aliado" property="og:title"/><meta content="Tiendas makro" property="og:description"/><meta content="Makro" property="og:site_name"/><meta content="https://www.makro.pe/" property="og:url"/><meta content="https://www.makro.pe/opengraph/default-image.jpeg" property="og:image"/><meta content="800" property="og:image:width"/><meta content="600" property="og:image:height"/><meta content="Makro" property="og:image:alt"/><meta content="summary_large_image" name="twitter:card"/><meta content="Makro" name="twitter:site"/><meta content="Makro" name="twitter:creator"/><meta cont

In [4]:
#Revisamos la estructura de la página y buscamos el componente que guarde la información de las tiendas
# Si buscamos dentro del contenido podemos ver que en el componente que
#   <script id="__NEXT_DATA__" type="application/json" crossorigin="anonymous"> 
# Contiene un json el cual puede tener la información de las tiendas
script_div = soup.find(id="__NEXT_DATA__")
print(script_div)


<script crossorigin="anonymous" id="__NEXT_DATA__" type="application/json">{"props":{"pageProps":{"token":{"id":12964000,"token":"6683f662-216c-43cb-9ad2-35960d826282","data":"{'Host': 'api.makro.pe', 'X-Forwarded-Proto': 'http', 'X-Real-Ip': '172.23.134.238', 'Connection': 'close', 'Content-Length': '48', 'Accept': 'application/json, text/plain, */*', 'Content-Type': 'application/json', 'User-Agent': 'axios/0.26.1'}","store":null},"appearance":{"header":{"logo":{"image":"https://files.makro.pe/logo/image/logo_makro_yellow_whj2AdH.webp","alt":"makro","position":"Cabecera"},"menu":[{"target":"self","position":"Cabecera","url":"http://makro.pe/makroahorro","alias":"Makroahorro","submenu":[{"target":"self","url":"http://makro.pe/makroahorro/destacados","alias":"Destacados"},{"target":"self","url":"http://makro.pe/makroahorro/tarjeta-oh","alias":"Tarjeta Oh"},{"target":"self","url":"http://makro.pe/makroahorro/marcas-makro","alias":"Productos Makro"}]},{"target":"self","position":"Cabecera

In [5]:
# Obtenemos solo el contendio
json_contenido = script_div.getText()
json_contenido

'{"props":{"pageProps":{"token":{"id":12964000,"token":"6683f662-216c-43cb-9ad2-35960d826282","data":"{\'Host\': \'api.makro.pe\', \'X-Forwarded-Proto\': \'http\', \'X-Real-Ip\': \'172.23.134.238\', \'Connection\': \'close\', \'Content-Length\': \'48\', \'Accept\': \'application/json, text/plain, */*\', \'Content-Type\': \'application/json\', \'User-Agent\': \'axios/0.26.1\'}","store":null},"appearance":{"header":{"logo":{"image":"https://files.makro.pe/logo/image/logo_makro_yellow_whj2AdH.webp","alt":"makro","position":"Cabecera"},"menu":[{"target":"self","position":"Cabecera","url":"http://makro.pe/makroahorro","alias":"Makroahorro","submenu":[{"target":"self","url":"http://makro.pe/makroahorro/destacados","alias":"Destacados"},{"target":"self","url":"http://makro.pe/makroahorro/tarjeta-oh","alias":"Tarjeta Oh"},{"target":"self","url":"http://makro.pe/makroahorro/marcas-makro","alias":"Productos Makro"}]},{"target":"self","position":"Cabecera","url":"http://makro.pe/catalogos","alias

In [6]:
# Podemos pasarlo a una estructa de diccionario para manejarlo en python
dict_contenido = json.loads(json_contenido)
dict_contenido

{'props': {'pageProps': {'token': {'id': 12964000,
    'token': '6683f662-216c-43cb-9ad2-35960d826282',
    'data': "{'Host': 'api.makro.pe', 'X-Forwarded-Proto': 'http', 'X-Real-Ip': '172.23.134.238', 'Connection': 'close', 'Content-Length': '48', 'Accept': 'application/json, text/plain, */*', 'Content-Type': 'application/json', 'User-Agent': 'axios/0.26.1'}",
    'store': None},
   'appearance': {'header': {'logo': {'image': 'https://files.makro.pe/logo/image/logo_makro_yellow_whj2AdH.webp',
      'alt': 'makro',
      'position': 'Cabecera'},
     'menu': [{'target': 'self',
       'position': 'Cabecera',
       'url': 'http://makro.pe/makroahorro',
       'alias': 'Makroahorro',
       'submenu': [{'target': 'self',
         'url': 'http://makro.pe/makroahorro/destacados',
         'alias': 'Destacados'},
        {'target': 'self',
         'url': 'http://makro.pe/makroahorro/tarjeta-oh',
         'alias': 'Tarjeta Oh'},
        {'target': 'self',
         'url': 'http://makro.pe/m

In [7]:
# La lista que necesitamos se encuentra dentro de "stores"
lista_tiendas = dict_contenido["props"]["pageProps"]["pageData"]["stores"]
lista_tiendas

[{'id': 29,
  'code': '27',
  'name': 'Makro Cercado de Lima',
  'manager': '.',
  'email': 'contacto@makro.pe',
  'address': 'Jr. Manuel del Mar y Bernedo S/N\xa0Lote\xa0A',
  'latitud': None,
  'longitud': None,
  'province': None,
  'phone': None,
  'attention': ['Lunes a Domingo de 7:00 am a 9:30 pm'],
  'og_description': None,
  'og_image': None},
 {'id': 28,
  'code': '26',
  'name': 'Makro Chimbote',
  'manager': '.',
  'email': 'contacto@makro.pe',
  'address': 'Av. Los pescadores s/n zona lotización Gran Trapecio Parcela 3, Mz. C.',
  'latitud': None,
  'longitud': None,
  'province': None,
  'phone': None,
  'attention': ['Lunes a Domingo de 7:00 am. a 10:00 pm.'],
  'og_description': None,
  'og_image': None},
 {'id': 27,
  'code': '25',
  'name': 'Makro San Martin de Porres',
  'manager': '.',
  'email': 'contacto@makro.pe',
  'address': 'PLAZA CENTER REX',
  'latitud': None,
  'longitud': None,
  'province': None,
  'phone': None,
  'attention': ['Lunes a Domingo de 7:30 a

In [8]:
# necesitamos los campos de name ("nombre de tienda") y address ("direccion de tienda")
for tienda in lista_tiendas:
    print(f"Nombre de la tienda: {tienda['name']} - Dirección: {tienda['address']}")

Nombre de la tienda: Makro Cercado de Lima - Dirección: Jr. Manuel del Mar y Bernedo S/N Lote A
Nombre de la tienda: Makro Chimbote - Dirección: Av. Los pescadores s/n zona lotización Gran Trapecio Parcela 3, Mz. C.
Nombre de la tienda: Makro San Martin de Porres - Dirección: PLAZA CENTER REX
Nombre de la tienda: Makro Cañete - Dirección: Parcela 6 del proyecto Chilca - San Vicente de Cañete
Nombre de la tienda: Makro Huacho - Dirección: AV. PANAMERICANA CRUCE CON AV. SAN MARTIN – SANTA MARIA, HUACHO, LIMA
Nombre de la tienda: Makro Ica - Dirección: AV. NICOLÁS DE RIVERO EL VIEJO 1105 - ICA
Nombre de la tienda: Makro Chincha - Dirección: CARRETERA PANAMERICANA SUR KM. 195 - Chincha
Nombre de la tienda: Makro Huancayo - Dirección: AV. PROLONGACIÓN JOSÉ CARLOS MARIATEGUI S/N, HUANCAYO, HUANCAYO
Nombre de la tienda: Makro Sullana - Dirección: ZONA INDUSTRIAL MUNICIPAL N2 MZE Lt2
Nombre de la tienda: Makro Textil - Piura - Dirección: AV. SÁNCHEZ CERRO SUB LOTE B, 1A DISTRITO 26 DE OCTUBRE


In [9]:
lista_direcciones=[d["address"] for d in lista_tiendas]
lista_direcciones

['Jr. Manuel del Mar y Bernedo S/N\xa0Lote\xa0A',
 'Av. Los pescadores s/n zona lotización Gran Trapecio Parcela 3, Mz. C.',
 'PLAZA CENTER REX',
 'Parcela 6 del proyecto Chilca - San Vicente de Cañete',
 'AV. PANAMERICANA CRUCE CON AV. SAN MARTIN – SANTA MARIA, HUACHO, LIMA',
 'AV. NICOLÁS DE RIVERO EL VIEJO 1105 - ICA',
 'CARRETERA PANAMERICANA SUR KM. 195 - Chincha',
 'AV. PROLONGACIÓN JOSÉ CARLOS MARIATEGUI S/N, HUANCAYO, HUANCAYO',
 'ZONA INDUSTRIAL MUNICIPAL N2 MZE Lt2',
 'AV. SÁNCHEZ CERRO SUB LOTE B, 1A DISTRITO 26 DE OCTUBRE',
 'AV. SANCHEZ CERRO S/N',
 'AV. INDUSTRIAL CRUCE CON AV. FEDERICO VILLAREAL – EL BOSQUE, TRUJILLO,',
 'AV. NICOLAS DE PIEROLA CON AV. MICAELA BASTIDAS , TRUJILLO',
 'AV. JORGE BASADRE #299 ,CHICLAYO',
 'PROLONGACIÓN AV DE LA CULTURA 2620 CUSCO',
 'AV. METROPOLITANA 63, DISTRITO YANAHUARA, AREQUIPA',
 'AV. AVELINO CACERES S/N , DISTRITO DE BUSTAMANTE Y RIVERO, AREQUIPA',
 'AV. CHACRA CERRO S/N LOTE 115, LIMA',
 'AV. GRAN PAJATÉN CRUCE CON JR. SANTUARIO, L

In [10]:
lista_nombres=[d["name"] for d in lista_tiendas]
lista_nombres

['Makro Cercado de Lima',
 'Makro Chimbote',
 'Makro San Martin de Porres',
 'Makro Cañete',
 'Makro Huacho',
 'Makro Ica',
 'Makro Chincha',
 'Makro Huancayo',
 'Makro Sullana',
 'Makro Textil - Piura',
 'Makro Sánchez Cerro - Piura',
 'Makro El Bosque - Trujillo',
 'Makro La Esperanza - Trujillo',
 'Makro Chiclayo',
 'Makro Cusco',
 'Makro Yanahuara - Arequipa',
 'Makro El Avelino - Arequipa',
 'Makro Comas',
 'Makro San Juan de Lurigancho',
 'Makro Zárate',
 'Makro Chorrillos',
 'Makro Villa el Salvador',
 'Makro Independencia',
 'Makro Santa Anita',
 'Makro Surco',
 'Makro Callao']

In [11]:
def get_driver():
    chrome_options = Options()
    #chrome_options.add_argument("--headless")
    prefs = {"profile.default_content_setting_values.notifications" : 2}
    chrome_options.add_experimental_option("prefs",prefs)
    s = ChromeService("chromedriver.exe")
    driver = webdriver.Chrome(service=s, options=chrome_options)
    driver.maximize_window()
    return driver

In [12]:
list_direcciones_final = []

driver = get_driver()

for  direccion in tqdm.tqdm(lista_direcciones):

    dir = direccion[1:-1] + ", Peru, Makro"
    #dir = dir.replace('/', ' ').replace('#', ' ').replace('.', ' ')

    #driver.get("https://www.google.com/maps/search/"+dir)

    driver.get("https://www.google.com/maps/")

    searchbox = driver.find_element(By.ID, 'searchboxinput')

    searchbox.send_keys(dir)

    searchbutton = driver.find_element(By.ID, 'searchbox-searchbutton')

    searchbutton.click()

    time.sleep(3)

    try:
        wait(driver, 4).until(ec.visibility_of_element_located((By.XPATH, "html/body/div[@id='app-container']/div[@id='content-container']/div[@id='QA0Szd']/div/div[@class='XltNde tTVLSc']/div[@class='w6VYqd']/div[@class='bJzME tTVLSc']/div[@class='k7jAl lJ3Kh miFGmb']/div[@tabindex='-1']/div[@class='aIFcqe']/div[@class='m6QErb WNBkOb ']/div[@class='TIHn2 ']/div[@class='tAiQdd']/div[@class='lMbq3e']")))
        coordenadas = driver.find_element(By.CSS_SELECTOR, 'meta[itemprop=image]').get_attribute('content')

        latitud = float(coordenadas[coordenadas.find('-'):coordenadas.find("%")])
        longitud =float(coordenadas[coordenadas.find('-', coordenadas.find("-")+1):coordenadas.find("&")])

        imagen = driver.find_element(By.CSS_SELECTOR, "button.aoRNLd.kn2E5e.NMjTrf.lvtCsd > img").get_attribute('src')

        nombre_maps = driver.find_element(By.CSS_SELECTOR, "h1.DUwDvf.lfPIob").text

        direccion_maps = driver.find_element(By.CLASS_NAME, "CsEnBe").text

        comentario = 'Dirección encontrada'
    
    except:

        try:
            wait(driver, 4).until(ec.element_to_be_clickable((By.CLASS_NAME, "hfpxzc")))
            boton = driver.find_element(By.CLASS_NAME, "hfpxzc")
            boton.click()

            time.sleep(3)
            coordenadas = driver.find_element(By.CSS_SELECTOR, 'meta[itemprop=image]').get_attribute('content')

            latitud = float(coordenadas[coordenadas.find('-'):coordenadas.find("%")])
            longitud =float(coordenadas[coordenadas.find('-', coordenadas.find("-")+1):coordenadas.find("&")])

            imagen = driver.find_element(By.CSS_SELECTOR, "button.aoRNLd.kn2E5e.NMjTrf.lvtCsd > img").get_attribute('src')

            nombre_maps = driver.find_element(By.CSS_SELECTOR, "h1.DUwDvf.lfPIob").text

            comentario = 'Más de una dirección encontrada, toma la primera'

            try:
                direccion_maps = driver.find_element(By.CLASS_NAME, "CsEnBe").text
                
            except:

                try:
                    direccion_maps = driver.find_element(By.CSS_SELECTOR, "span.DkEaL").text
                
                except:
                    direccion_maps = ''
                    comentario = "Dirección no encontrada"
            
            print(comentario)

        except:
            
            comentario = "Dirección no encontrada"
            print(comentario)
        
            latitud = np.nan
            longitud = np.nan

            imagen = ''

            nombre_maps = ''

            direccion_maps = ''
            
            driver.get("https://www.google.com/maps")

    list_direcciones_final.append([direccion,imagen, latitud, longitud, comentario, nombre_maps, direccion_maps])

driver.close()

SessionNotCreatedException: Message: session not created: This version of ChromeDriver only supports Chrome version 114
Current browser version is 123.0.6312.88 with binary path C:\Program Files\Google\Chrome\Application\chrome.exe
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00A6A813+48355]
	(No symbol) [0x009FC4B1]
	(No symbol) [0x00905358]
	(No symbol) [0x009261AC]
	(No symbol) [0x00921EF3]
	(No symbol) [0x00920579]
	(No symbol) [0x00950C55]
	(No symbol) [0x0095093C]
	(No symbol) [0x0094A536]
	(No symbol) [0x009282DC]
	(No symbol) [0x009293DD]
	GetHandleVerifier [0x00CCAABD+2539405]
	GetHandleVerifier [0x00D0A78F+2800735]
	GetHandleVerifier [0x00D0456C+2775612]
	GetHandleVerifier [0x00AF51E0+616112]
	(No symbol) [0x00A05F8C]
	(No symbol) [0x00A02328]
	(No symbol) [0x00A0240B]
	(No symbol) [0x009F4FF7]
	BaseThreadInitThunk [0x76A07BA9+25]
	RtlInitializeExceptionChain [0x7731BDAB+107]
	RtlClearBits [0x7731BD2F+191]


In [ ]:
df_tiendas_makro = pd.DataFrame(list_direcciones_final, columns=[ 'direccion', 'imagen', 'latitud', 'longitud', 'comentario', 'nombre_maps', 'direccion_maps'])
df_tiendas_makro

,direccion,imagen,latitud,longitud,comentario,nombre_maps,direccion_maps
0,Jr. Manuel del Mar y Bernedo S/N Lote A,https://lh5.googleusercontent.com/p/AF1QipO5rr...,-6.766592,-79.865446,"Más de una dirección encontrada, toma la primera",Makro,"Carr. Central 1, Lima 15008"
1,Av. Los pescadores s/n zona lotización Gran Tr...,,NaN,NaN,Dirección no encontrada,,
2,PLAZA CENTER REX,https://lh5.googleusercontent.com/p/AF1QipNQQX...,-6.761074,-79.863283,"Más de una dirección encontrada, toma la primera",Makro El Bosque Trujillo,"V2Q3+W8M, AV. INDUSTRIAL MZA. W, Laredo"
3,Parcela 6 del proyecto Chilca - San Vicente de...,https://lh5.googleusercontent.com/p/AF1QipPm53...,-6.761074,-79.863283,Dirección no encontrada,Chilca,
4,AV. PANAMERICANA CRUCE CON AV. SAN MARTIN – SA...,https://lh5.googleusercontent.com/p/AF1QipMHdB...,-6.761074,-79.863283,Dirección encontrada,Makro Huacho,"Ave San Martin 580, Huacho 15138"
5,AV. NICOLÁS DE RIVERO EL VIEJO 1105 - ICA,,NaN,NaN,Dirección no encontrada,,
6,CARRETERA PANAMERICANA SUR KM. 195 - Chincha,https://lh5.googleusercontent.com/p/AF1QipOwhs...,-6.761074,-79.863283,Dirección encontrada,Makro Chincha,Panamericana Sur KM 195
7,"AV. PROLONGACIÓN JOSÉ CARLOS MARIATEGUI S/N, H...",https://lh5.googleusercontent.com/p/AF1QipOVtL...,-6.761074,-79.863283,Dirección encontrada,Makro ¡Tu mejor Aliado! - Huancayo,"Av. Prolongación José Caros Mariátegui, Huanca..."
8,ZONA INDUSTRIAL MUNICIPAL N2 MZE Lt2,,NaN,NaN,Dirección no encontrada,,
9,"AV. SÁNCHEZ CERRO SUB LOTE B, 1A DISTRITO 26 D...",,NaN,NaN,Dirección no encontrada,,


In [ ]:
df_tiendas_makro['COMPETIDOR']="Makro"

In [ ]:
df_nombre = pd.DataFrame(lista_nombres)

In [ ]:
df_seleccion_Makro = df_tiendas_makro[['COMPETIDOR','direccion','latitud', 'longitud']]
df_seleccion_Makro

,COMPETIDOR,direccion,latitud,longitud
0,Makro,Jr. Manuel del Mar y Bernedo S/N Lote A,-6.766592,-79.865446
1,Makro,Av. Los pescadores s/n zona lotización Gran Tr...,NaN,NaN
2,Makro,PLAZA CENTER REX,-6.761074,-79.863283
3,Makro,Parcela 6 del proyecto Chilca - San Vicente de...,-6.761074,-79.863283
4,Makro,AV. PANAMERICANA CRUCE CON AV. SAN MARTIN – SA...,-6.761074,-79.863283
5,Makro,AV. NICOLÁS DE RIVERO EL VIEJO 1105 - ICA,NaN,NaN
6,Makro,CARRETERA PANAMERICANA SUR KM. 195 - Chincha,-6.761074,-79.863283
7,Makro,"AV. PROLONGACIÓN JOSÉ CARLOS MARIATEGUI S/N, H...",-6.761074,-79.863283
8,Makro,ZONA INDUSTRIAL MUNICIPAL N2 MZE Lt2,NaN,NaN
9,Makro,"AV. SÁNCHEZ CERRO SUB LOTE B, 1A DISTRITO 26 D...",NaN,NaN


In [ ]:
resultado = pd.concat([df_nombre,df_seleccion_Makro], ignore_index=True,axis=1)
re=resultado[[1, 0, 2, 3, 4]]
re

,1,0,2,3,4
0,Makro,Makro Cercado de Lima,Jr. Manuel del Mar y Bernedo S/N Lote A,-6.766592,-79.865446
1,Makro,Makro Chimbote,Av. Los pescadores s/n zona lotización Gran Tr...,NaN,NaN
2,Makro,Makro San Martin de Porres,PLAZA CENTER REX,-6.761074,-79.863283
3,Makro,Makro Cañete,Parcela 6 del proyecto Chilca - San Vicente de...,-6.761074,-79.863283
4,Makro,Makro Huacho,AV. PANAMERICANA CRUCE CON AV. SAN MARTIN – SA...,-6.761074,-79.863283
5,Makro,Makro Ica,AV. NICOLÁS DE RIVERO EL VIEJO 1105 - ICA,NaN,NaN
6,Makro,Makro Chincha,CARRETERA PANAMERICANA SUR KM. 195 - Chincha,-6.761074,-79.863283
7,Makro,Makro Huancayo,"AV. PROLONGACIÓN JOSÉ CARLOS MARIATEGUI S/N, H...",-6.761074,-79.863283
8,Makro,Makro Sullana,ZONA INDUSTRIAL MUNICIPAL N2 MZE Lt2,NaN,NaN
9,Makro,Makro Textil - Piura,"AV. SÁNCHEZ CERRO SUB LOTE B, 1A DISTRITO 26 D...",NaN,NaN


In [ ]:
df_Makro = re.rename(columns={0:"NOMBRE_LOCAL",1:"COMPETIDOR",2:"DIRECCIÓN",3:"LATITUD",4:"LONGITUD"})
df_Makro

,COMPETIDOR,NOMBRE_LOCAL,DIRECCIÓN,LATITUD,LONGITUD
0,Makro,Makro Cercado de Lima,Jr. Manuel del Mar y Bernedo S/N Lote A,-6.766592,-79.865446
1,Makro,Makro Chimbote,Av. Los pescadores s/n zona lotización Gran Tr...,NaN,NaN
2,Makro,Makro San Martin de Porres,PLAZA CENTER REX,-6.761074,-79.863283
3,Makro,Makro Cañete,Parcela 6 del proyecto Chilca - San Vicente de...,-6.761074,-79.863283
4,Makro,Makro Huacho,AV. PANAMERICANA CRUCE CON AV. SAN MARTIN – SA...,-6.761074,-79.863283
5,Makro,Makro Ica,AV. NICOLÁS DE RIVERO EL VIEJO 1105 - ICA,NaN,NaN
6,Makro,Makro Chincha,CARRETERA PANAMERICANA SUR KM. 195 - Chincha,-6.761074,-79.863283
7,Makro,Makro Huancayo,"AV. PROLONGACIÓN JOSÉ CARLOS MARIATEGUI S/N, H...",-6.761074,-79.863283
8,Makro,Makro Sullana,ZONA INDUSTRIAL MUNICIPAL N2 MZE Lt2,NaN,NaN
9,Makro,Makro Textil - Piura,"AV. SÁNCHEZ CERRO SUB LOTE B, 1A DISTRITO 26 D...",NaN,NaN


In [ ]:
df_Makro.to_csv('Makro.csv',index=False)